# Modelo lineal para la prediccion


In [104]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer

In [63]:
df = pd.read_csv("data/clean/personas_limpio.csv")

In [64]:
def convert_to_categorical(df, columns):
    """
    Convierte las columnas especificadas de un DataFrame a tipo categórico.

    Parámetros:
    df (pd.DataFrame): El DataFrame en el que se realizarán las conversiones.
    columns (list): Lista de nombres de columnas a convertir a tipo categórico.

    Retorna:
    pd.DataFrame: El DataFrame con las columnas especificadas convertidas a tipo categórico.
    """
    for column in columns:
        if column in df.columns:
            df[column] = pd.Categorical(df[column]).codes
        else:
            print(f"Columna '{column}' no encontrada en el DataFrame.")
    return df

categorical_columns = ['gender', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'smoking_status','stroke']
df = convert_to_categorical(df, categorical_columns)

## Caso base

In [80]:
X = df.drop(columns=["stroke"])
y = df[["stroke"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# import mlflow
# import mlflow.sklearn

# experiment = mlflow.set_experiment("project_linear_model")

# with mlflow.start_run(run_name="modelo_base"):
#     # Creating a pipeline that standardizes data and then applies logistic regression
#     penalty = "l2"
#     class_weight = None
#     max_iter = 100


#     pipeline = Pipeline([
#         ('scaler', StandardScaler()),  # Step 1: Standardize the numeric features
#         ('logreg', LogisticRegression(penalty=penalty, 
#                                     class_weight=class_weight,
#                                     max_iter=max_iter))  # Step 2: Apply Logistic Regression
#     ])

#     # Fitting the model on the training data
#     pipeline.fit(X_train, y_train)

#     # Predicting on the test data
#     y_pred = pipeline.predict(X_test)


#     # registrar parametros
#     mlflow.log_param("penalty", penalty)
#     mlflow.log_param("class_weight", class_weight)
#     mlflow.log_param("max_iter", max_iter)

#     # registrar el modelo
#     mlflow.sklearn.log_model(pipeline, "logistic_regression_base")


#     # Evaluating the model
#     accuracy = accuracy_score(y_test, y_pred)
#     print(f"Accuracy: {accuracy:.2f}")

#     f1_scr = f1_score(y_test, y_pred)
#     print(f"f1_score: {f1_scr:.2f}")

#     conf_mtrx = confusion_matrix(y_test,y_pred)
#     print(f"confusion matrix: {f1_scr:.2f}")

#     mlflow.log_metric("accuracy",accuracy)    
#     mlflow.log_metric("f1_score",f1_scr)
#     mlflow.log_metric("confusion_matrix",conf_mtrx) 

In [ ]:
# experiment = mlflow.set_experiment("project_linear_model")

# with mlflow.start_run(run_name="modelo_balanced"):
#     # Creating a pipeline that standardizes data and then applies logistic regression
#     penalty = "l2"
#     class_weight = "balanced"
#     max_iter = 100


#     pipeline = Pipeline([
#         ('scaler', StandardScaler()),  # Step 1: Standardize the numeric features
#         ('logreg', LogisticRegression(penalty=penalty, 
#                                     class_weight=class_weight,
#                                     max_iter=max_iter))  # Step 2: Apply Logistic Regression
#     ])

#     # Fitting the model on the training data
#     pipeline.fit(X_train, y_train)

#     # Predicting on the test data
#     y_pred = pipeline.predict(X_test)


#     # registrar parametros
#     mlflow.log_param("penalty", penalty)
#     mlflow.log_param("class_weight", class_weight)
#     mlflow.log_param("max_iter", max_iter)

#     # registrar el modelo
#     mlflow.sklearn.log_model(pipeline, "logistic_regression_base")


#     # Evaluating the model
#     accuracy = accuracy_score(y_test, y_pred)
#     print(f"Accuracy: {accuracy:.2f}")

#     f1_scr = f1_score(y_test, y_pred)
#     print(f"f1_score: {f1_scr:.2f}")

#     conf_mtrx = confusion_matrix(y_test,y_pred)
#     print(f"confusion matrix: {f1_scr:.2f}")

#     mlflow.log_metric("accuracy",accuracy)    
#     mlflow.log_metric("f1_score",f1_scr)
#     mlflow.log_metric("confusion_matrix",conf_mtrx)

In [93]:

# Creating a pipeline that standardizes data and then applies logistic regression
penalty = "l2"
class_weight = None
max_iter = 100


param_grid = {"logreg__penalty": ["l2","l1","elasticnet",None],
              "logreg__class_weight":[None, "balanced"],
              "logreg__max_iter": np.arange(100,200,20)
              }
              #"poly__degree": [1, 2, 3]}

pipeline = Pipeline([
    #('poly', PolynomialFeatures(include_bias=False)),
    ('scaler', StandardScaler()),  # Step 1: Standardize the numeric features
    ('logreg', LogisticRegression(penalty=penalty, 
                                  class_weight=class_weight,
                                  max_iter=max_iter))  # Step 2: Apply Logistic Regression
])

# Fitting the model on the training data
pipeline.fit(X_train, y_train)

# Predicting on the test data
y_pred = pipeline.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

f1_scr = f1_score(y_test, y_pred)
print(f"f1_score: {f1_scr:.2f}")

confusion_matrix(y_test, y_pred)

Accuracy: 0.94
f1_score: 0.00


c:\Users\Paul\anaconda3\envs\tf\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[960,   0],
       [ 62,   0]], dtype=int64)

In [98]:
grid_search = GridSearchCV(pipeline, param_grid, scoring='f1', cv=5, verbose=3)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV 1/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2;, score=0.000 total time=   0.0s
[CV 2/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2;, score=0.000 total time=   0.0s
[CV 3/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2;, score=0.000 total time=   0.0s
[CV 4/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2;, score=0.000 total time=   0.0s
[CV 5/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2;, score=0.000 total time=   0.0s
[CV 1/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l1;, score=nan total time=   0.0s
[CV 2/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l1;, score=nan total time=   0.0s
[CV 3/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l1;, score=nan total time=   0.0s
[CV 4/5] END logreg__class_weigh

c:\Users\Paul\anaconda3\envs\tf\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Paul\anaconda3\envs\tf\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Paul\anaconda3\envs\tf\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Paul\anaconda3\envs\tf\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

[CV 3/5] END logreg__class_weight=None, logreg__max_iter=120, logreg__penalty=l2;, score=0.000 total time=   0.0s
[CV 4/5] END logreg__class_weight=None, logreg__max_iter=120, logreg__penalty=l2;, score=0.000 total time=   0.0s
[CV 5/5] END logreg__class_weight=None, logreg__max_iter=120, logreg__penalty=l2;, score=0.000 total time=   0.0s
[CV 1/5] END logreg__class_weight=None, logreg__max_iter=120, logreg__penalty=l1;, score=nan total time=   0.0s
[CV 2/5] END logreg__class_weight=None, logreg__max_iter=120, logreg__penalty=l1;, score=nan total time=   0.0s
[CV 3/5] END logreg__class_weight=None, logreg__max_iter=120, logreg__penalty=l1;, score=nan total time=   0.0s
[CV 4/5] END logreg__class_weight=None, logreg__max_iter=120, logreg__penalty=l1;, score=nan total time=   0.0s
[CV 5/5] END logreg__class_weight=None, logreg__max_iter=120, logreg__penalty=l1;, score=nan total time=   0.0s
[CV 1/5] END logreg__class_weight=None, logreg__max_iter=120, logreg__penalty=elasticnet;, score=n

c:\Users\Paul\anaconda3\envs\tf\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Paul\anaconda3\envs\tf\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Paul\anaconda3\envs\tf\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Paul\anaconda3\envs\tf\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

[CV 5/5] END logreg__class_weight=None, logreg__max_iter=140, logreg__penalty=l2;, score=0.000 total time=   0.0s
[CV 1/5] END logreg__class_weight=None, logreg__max_iter=140, logreg__penalty=l1;, score=nan total time=   0.0s
[CV 2/5] END logreg__class_weight=None, logreg__max_iter=140, logreg__penalty=l1;, score=nan total time=   0.0s
[CV 3/5] END logreg__class_weight=None, logreg__max_iter=140, logreg__penalty=l1;, score=nan total time=   0.0s
[CV 4/5] END logreg__class_weight=None, logreg__max_iter=140, logreg__penalty=l1;, score=nan total time=   0.0s
[CV 5/5] END logreg__class_weight=None, logreg__max_iter=140, logreg__penalty=l1;, score=nan total time=   0.0s
[CV 1/5] END logreg__class_weight=None, logreg__max_iter=140, logreg__penalty=elasticnet;, score=nan total time=   0.0s
[CV 2/5] END logreg__class_weight=None, logreg__max_iter=140, logreg__penalty=elasticnet;, score=nan total time=   0.0s
[CV 3/5] END logreg__class_weight=None, logreg__max_iter=140, logreg__penalty=elasticn

...

[CV 1/5] END logreg__class_weight=balanced, logreg__max_iter=140, logreg__penalty=l2;, score=0.186 total time=   0.0s
[CV 2/5] END logreg__class_weight=balanced, logreg__max_iter=140, logreg__penalty=l2;, score=0.237 total time=   0.0s
[CV 3/5] END logreg__class_weight=balanced, logreg__max_iter=140, logreg__penalty=l2;, score=0.208 total time=   0.0s
[CV 4/5] END logreg__class_weight=balanced, logreg__max_iter=140, logreg__penalty=l2;, score=0.239 total time=   0.0s
[CV 5/5] END logreg__class_weight=balanced, logreg__max_iter=140, logreg__penalty=l2;, score=0.240 total time=   0.0s
[CV 1/5] END logreg__class_weight=balanced, logreg__max_iter=140, logreg__penalty=l1;, score=nan total time=   0.0s
[CV 2/5] END logreg__class_weight=balanced, logreg__max_iter=140, logreg__penalty=l1;, score=nan total time=   0.0s
[CV 3/5] END logreg__class_weight=balanced, logreg__max_iter=140, logreg__penalty=l1;, score=nan total time=   0.0s
[CV 4/5] END logreg__class_weight=balanced, logreg__max_iter=1

c:\Users\Paul\anaconda3\envs\tf\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Paul\anaconda3\envs\tf\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Paul\anaconda3\envs\tf\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Paul\anaconda3\envs\tf\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

[CV 1/5] END logreg__class_weight=balanced, logreg__max_iter=160, logreg__penalty=l2;, score=0.186 total time=   0.0s
[CV 2/5] END logreg__class_weight=balanced, logreg__max_iter=160, logreg__penalty=l2;, score=0.237 total time=   0.0s
[CV 3/5] END logreg__class_weight=balanced, logreg__max_iter=160, logreg__penalty=l2;, score=0.208 total time=   0.0s
[CV 4/5] END logreg__class_weight=balanced, logreg__max_iter=160, logreg__penalty=l2;, score=0.239 total time=   0.0s
[CV 5/5] END logreg__class_weight=balanced, logreg__max_iter=160, logreg__penalty=l2;, score=0.240 total time=   0.0s
[CV 1/5] END logreg__class_weight=balanced, logreg__max_iter=160, logreg__penalty=l1;, score=nan total time=   0.0s
[CV 2/5] END logreg__class_weight=balanced, logreg__max_iter=160, logreg__penalty=l1;, score=nan total time=   0.0s
[CV 3/5] END logreg__class_weight=balanced, logreg__max_iter=160, logreg__penalty=l1;, score=nan total time=   0.0s
[CV 4/5] END logreg__class_weight=balanced, logreg__max_iter=1

c:\Users\Paul\anaconda3\envs\tf\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Paul\anaconda3\envs\tf\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Paul\anaconda3\envs\tf\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Paul\anaconda3\envs\tf\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('logreg', LogisticRegression())]),
             param_grid={'logreg__class_weight': [None, 'balanced'],
                         'logreg__max_iter': array([100, 120, 140, 160, 180]),
                         'logreg__penalty': ['l2', 'l1', 'elasticnet', None]},
             scoring='f1', verbose=3)

In [99]:
grid_search.best_score_

0.22206706383552036

In [100]:
grid_search.best_params_

{'logreg__class_weight': 'balanced',
 'logreg__max_iter': 100,
 'logreg__penalty': 'l2'}

array([0, 0, 0, ..., 0, 1, 1], dtype=int8)

In [107]:
poly_columns = df.select_dtypes("float").columns.values
not_poly_columns = [x for x in df.columns if x not in poly_columns]
not_poly_columns

['gender',
 'hypertension',
 'heart_disease',
 'ever_married',
 'work_type',
 'Residence_type',
 'smoking_status',
 'stroke']

In [121]:
poly_columns = X_train.select_dtypes("float").columns.values
not_poly_columns = [x for x in X_train.columns if x not in poly_columns]

preprocessor = ColumnTransformer(
    transformers=[
        ('poly', PolynomialFeatures(include_bias=False), poly_columns),
        ('pass', 'passthrough', not_poly_columns)
    ]
)



# Creating a pipeline that standardizes data and then applies logistic regression
penalty = "l2"
class_weight = None
max_iter = 100


param_grid = {"logreg__penalty": ["l2","l1","elasticnet",None],
              "logreg__class_weight":[None, "balanced"],
              "logreg__max_iter": np.arange(100,400,50),
              "poly__poly__degree": [1, 2, 3, 4]}

pipeline = Pipeline([
    ('poly', preprocessor),
    ('scaler', StandardScaler()),  # Step 1: Standardize the numeric features
    ('logreg', LogisticRegression(penalty=penalty, 
                                  class_weight=class_weight,
                                  max_iter=max_iter))  # Step 2: Apply Logistic Regression
])

# Fitting the model on the training data
pipeline.fit(X_train, y_train)

# Predicting on the test data
y_pred = pipeline.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

f1_scr = f1_score(y_test, y_pred)
print(f"f1_score: {f1_scr:.2f}")

confusion_matrix(y_test, y_pred)

Accuracy: 0.94
f1_score: 0.00


c:\Users\Paul\anaconda3\envs\tf\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[960,   0],
       [ 62,   0]], dtype=int64)

In [122]:
grid_search = GridSearchCV(pipeline, param_grid, scoring='f1', cv=5, verbose=3)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 192 candidates, totalling 960 fits
[CV 1/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2, poly__poly__degree=1;, score=0.000 total time=   0.0s
[CV 2/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2, poly__poly__degree=1;, score=0.000 total time=   0.0s
[CV 3/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2, poly__poly__degree=1;, score=0.000 total time=   0.0s
[CV 4/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2, poly__poly__degree=1;, score=0.000 total time=   0.0s
[CV 5/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2, poly__poly__degree=1;, score=0.000 total time=   0.0s
[CV 1/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2, poly__poly__degree=2;, score=0.000 total time=   0.0s
[CV 2/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2, poly__poly__degree=2;, score=0.000 tota

c:\Users\Paul\anaconda3\envs\tf\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Paul\anaconda3\envs\tf\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Paul\anaconda3\envs\tf\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Paul\anaconda3\envs\tf\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

[CV 5/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2, poly__poly__degree=2;, score=0.000 total time=   0.0s
[CV 1/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2, poly__poly__degree=3;, score=0.000 total time=   0.0s
[CV 2/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2, poly__poly__degree=3;, score=0.000 total time=   0.0s
[CV 3/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2, poly__poly__degree=3;, score=0.000 total time=   0.0s
[CV 4/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2, poly__poly__degree=3;, score=0.000 total time=   0.0s
[CV 5/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2, poly__poly__degree=3;, score=0.000 total time=   0.0s


c:\Users\Paul\anaconda3\envs\tf\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Paul\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Paul\anaconda3\envs\tf\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sampl

[CV 1/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2, poly__poly__degree=4;, score=0.000 total time=   0.0s
[CV 2/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2, poly__poly__degree=4;, score=0.000 total time=   0.0s
[CV 3/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2, poly__poly__degree=4;, score=0.000 total time=   0.0s
[CV 4/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2, poly__poly__degree=4;, score=0.000 total time=   0.0s
[CV 5/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l2, poly__poly__degree=4;, score=0.000 total time=   0.0s
[CV 1/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l1, poly__poly__degree=1;, score=nan total time=   0.0s
[CV 2/5] END logreg__class_weight=None, logreg__max_iter=100, logreg__penalty=l1, poly__poly__degree=1;, score=nan total time=   0.0s
[CV 3/5] END logreg__class_weight=None, logreg__max_

...

[CV 5/5] END logreg__class_weight=balanced, logreg__max_iter=350, logreg__penalty=None, poly__poly__degree=3;, score=0.234 total time=   0.0s
[CV 1/5] END logreg__class_weight=balanced, logreg__max_iter=350, logreg__penalty=None, poly__poly__degree=4;, score=0.187 total time=   0.0s
[CV 2/5] END logreg__class_weight=balanced, logreg__max_iter=350, logreg__penalty=None, poly__poly__degree=4;, score=0.228 total time=   0.0s


c:\Users\Paul\anaconda3\envs\tf\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Paul\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Paul\anaconda3\envs\tf\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_sampl

[CV 3/5] END logreg__class_weight=balanced, logreg__max_iter=350, logreg__penalty=None, poly__poly__degree=4;, score=0.218 total time=   0.0s
[CV 4/5] END logreg__class_weight=balanced, logreg__max_iter=350, logreg__penalty=None, poly__poly__degree=4;, score=0.229 total time=   0.0s
[CV 5/5] END logreg__class_weight=balanced, logreg__max_iter=350, logreg__penalty=None, poly__poly__degree=4;, score=0.249 total time=   0.0s


c:\Users\Paul\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Paul\anaconda3\envs\tf\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
480 fits failed out of a total of 960.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
240 fits failed with the f

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('poly',
                                        ColumnTransformer(transformers=[('poly',
                                                                         PolynomialFeatures(include_bias=False),
                                                                         array(['age', 'avg_glucose_level', 'bmi'], dtype=object)),
                                                                        ('pass',
                                                                         'passthrough',
                                                                         ['gender',
                                                                          'hypertension',
                                                                          'heart_disease',
                                                                          'ever_married',
                                                                          'work_type',
                                                                          'Residence_type',
                                                                          'smoking_status'])])),
                                       ('scaler', StandardScaler()),
                                       ('logreg', LogisticRegression())]),
             param_grid={'logreg__class_weight': [None, 'balanced'],
                         'logreg__max_iter': array([100, 150, 200, 250, 300, 350]),
                         'logreg__penalty': ['l2', 'l1', 'elasticnet', None],
                         'poly__poly__degree': [1, 2, 3, 4]},
             scoring='f1', verbose=3)

In [123]:
grid_search.best_score_

0.2266900128480386

In [124]:
grid_search.best_params_

{'logreg__class_weight': 'balanced',
 'logreg__max_iter': 300,
 'logreg__penalty': None,
 'poly__poly__degree': 4}

In [117]:
# Evaluating the model

y_pred=grid_search.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

f1_scr = f1_score(y_test, y_pred)
print(f"f1_score: {f1_scr:.2f}")

Accuracy: 0.74
f1_score: 0.26


In [118]:
confusion_matrix(y_test, y_pred)

array([[716, 244],
       [ 17,  45]], dtype=int64)